# Hướng dẫn sử dụng Phi-4 Mini ONNX cho Gọi Hàm Song Song

Notebook này hướng dẫn cách sử dụng Phi-4 Mini với ONNX Runtime GenAI để thực hiện gọi hàm song song. Gọi hàm cho phép mô hình thông minh kích hoạt các công cụ và API bên ngoài dựa trên yêu cầu của người dùng.

## Tổng quan

Trong hướng dẫn này, bạn sẽ học cách:
- Cài đặt Phi-4 Mini với ONNX Runtime GenAI
- Định nghĩa các cấu trúc hàm cho việc đặt vé máy bay và khách sạn
- Sử dụng tạo nội dung có hướng dẫn với ngữ pháp Lark để tạo đầu ra có cấu trúc
- Thực hiện gọi hàm song song cho các tình huống đặt chuyến đi phức tạp

## Yêu cầu

Trước khi chạy notebook này, hãy đảm bảo bạn đã:
- Tải xuống mô hình Phi-4 Mini ONNX
- Cài đặt gói `onnxruntime-genai`
- Hiểu cơ bản về các khái niệm gọi hàm


## Bước 1: Nhập các thư viện cần thiết

Đầu tiên, chúng ta sẽ nhập các thư viện cần thiết để triển khai việc gọi hàm.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## Bước 2: Thiết lập và cấu hình mô hình

Bây giờ chúng ta sẽ cấu hình mô hình Phi-4 Mini ONNX. Hãy đảm bảo thay thế đường dẫn bằng thư mục mô hình thực tế của bạn.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## Bước 3: Cấu hình Tham số Tạo

Thiết lập các tham số tạo để kiểm soát hành vi đầu ra của mô hình. Những cài đặt này đảm bảo phản hồi mang tính quyết định và tập trung cho việc gọi hàm.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## Bước 4: Xác định các hàm có sẵn

Ở đây, chúng ta xác định các hàm mà trợ lý AI của chúng ta có thể gọi. Trong ví dụ này, chúng ta có hai hàm liên quan đến du lịch:
1. **booking_flight_tickets**: Dùng để đặt vé máy bay giữa các sân bay
2. **booking_hotels**: Dùng để đặt chỗ ở tại khách sạn

Các định nghĩa hàm tuân theo định dạng schema gọi hàm của OpenAI.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Bước 5: Các Hàm Hỗ Trợ Tạo Ngữ Pháp

Các hàm hỗ trợ này chuyển đổi định nghĩa hàm của chúng ta sang định dạng ngữ pháp Lark, được ONNX Runtime GenAI sử dụng để tạo hướng dẫn. Điều này đảm bảo mô hình xuất ra các lời gọi hàm hợp lệ theo đúng định dạng JSON.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## Bước 6: Kiểm tra Tạo Ngữ Pháp

Hãy kiểm tra các hàm tạo ngữ pháp của chúng ta để xem cách chúng chuyển đổi định nghĩa công cụ thành định dạng phù hợp.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## Bước 7: Chuẩn bị Lời nhắc Hệ thống và Bộ tạo

Bây giờ chúng ta sẽ tạo lời nhắc hệ thống để thông báo cho mô hình về các công cụ có sẵn và thiết lập bộ tạo với các tham số hướng dẫn tạo nội dung.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## Bước 8: Khởi tạo Generator với Hướng dẫn Tạo nội dung

Bây giờ chúng ta sẽ tạo generator với các tham số đã cấu hình và áp dụng ngữ pháp Lark để hướng dẫn quá trình tạo nội dung.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## Bước 9: Kiểm tra Gọi Hàm Song Song

Bây giờ hãy kiểm tra thiết lập của chúng ta với một tình huống đặt chuyến đi phức tạp yêu cầu gọi nhiều hàm.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Tạo Lệnh Gọi Hàm

Mô hình sẽ tạo các lệnh gọi hàm có cấu trúc dựa trên yêu cầu của người dùng. Nhờ vào việc tạo hướng dẫn, đầu ra sẽ ở định dạng JSON hợp lệ có thể thực thi trực tiếp.

**Kết quả mong đợi**: Mô hình nên tạo lệnh gọi hàm cho:
1. `booking_flight_tickets` với thông tin từ Bắc Kinh (PEK) đến Paris (CDG)
2. `booking_hotels` với thông tin chỗ ở tại Paris

Chạy ô bên dưới để xem quá trình tạo trực tiếp:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Kết luận

🎉 **Chúc mừng!** Bạn đã triển khai thành công việc gọi hàm song song với Phi-4 Mini sử dụng ONNX Runtime GenAI.

### Những điều bạn đã học được:

1. **Cài đặt mô hình**: Cách cấu hình Phi-4 Mini với ONNX Runtime GenAI  
2. **Định nghĩa hàm**: Cách định nghĩa các schema hàm để gọi hàm AI  
3. **Tạo đầu ra có hướng dẫn**: Cách sử dụng ngữ pháp Lark để tạo đầu ra có cấu trúc  
4. **Gọi hàm song song**: Cách xử lý các yêu cầu phức tạp cần nhiều lần gọi hàm  

### Lợi ích chính:

- ✅ **Đầu ra có cấu trúc**: Tạo đầu ra có hướng dẫn đảm bảo các cuộc gọi hàm JSON hợp lệ  
- ✅ **Xử lý song song**: Xử lý nhiều lần gọi hàm trong một yêu cầu duy nhất  
- ✅ **Hiệu suất cao**: ONNX Runtime cung cấp khả năng suy luận tối ưu  
- ✅ **Schema linh hoạt**: Dễ dàng thêm hoặc chỉnh sửa định nghĩa hàm  

### Tài nguyên:

- [Tài liệu Phi-4 Mini](https://huggingface.co/microsoft/Phi-4-mini-onnx)  
- [Tài liệu ONNX Runtime GenAI](https://onnxruntime.ai/docs/genai/)  
- [Thực hành tốt nhất về gọi hàm](https://platform.openai.com/docs/guides/function-calling)  



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, khuyến nghị sử dụng dịch vụ dịch thuật chuyên nghiệp bởi con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
